
# 04 — Agents Hands-On (Python)

This notebook gives small, runnable Python examples:

1. A basic reactive agent.
2. A tool-using agent with a simple calculator tool.


## 0. Setup

In [ ]:

!pip install langchain langchain-openai --quiet

from dataclasses import dataclass
from typing import Dict, Any, Callable
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


## 1. Reactive Agent

In [ ]:

class ReactiveAgent:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, question: str) -> str:
        prompt = f"You are a helpful assistant. Answer briefly.

Question: {question}"
        return self.llm.invoke(prompt).content

agent = ReactiveAgent(llm)
agent.run("What is an AI agent?")


## 2. Tool-Using Agent (Calculator)

In [ ]:

def calculator(expression: str) -> str:
    try:
        result = eval(expression, {"__builtins__": {}})
        return f"{expression} = {result}"
    except Exception as e:
        return f"Error: {e}"

@dataclass
class ToolDecision:
    tool_name: str
    args: Dict[str, Any]

class ToolAgent:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def decide(self, question: str) -> ToolDecision | None:
        q = question.lower()
        if any(op in q for op in ["+", "-", "*", "/"]) or "calculate" in q:
            expr = q.replace("calculate", "").strip()
            if not expr:
                expr = q
            return ToolDecision("calculator", {"expression": expr})
        return None

    def run(self, question: str) -> str:
        decision = self.decide(question)
        if decision:
            tool_result = calculator(**decision.args)
            prompt = f"""User question: {question}
Tool used: calculator
Tool result: {tool_result}

Explain the result to the user in simple terms."""
            return self.llm.invoke(prompt).content
        else:
            return self.llm.invoke(question).content

tool_agent = ToolAgent(llm)
tool_agent.run("Calculate 3 * (4 + 5)")
